In [1]:
import numpy as np
import pyDR as DR
from pyDR.Project import Project
import matplotlib.pyplot as plt
from scipy.optimize import least_squares
import pickle
import os

In [2]:
ile_resn = [3, 13, 23, 30, 36, 44, 61]
#iles = [13, 61]
test_list = ["test_001", "test_002", "test_003", "test_010", "test_011", "test_014", "test_015", "test_charm"]

In [3]:
dir = "/localhome/mlehner/nmr/1ubq/"
#dir = "/home/mlehner/pyDR/1ubq_data/"
pdb_file_path = f"{dir}/1ubq_frame.pdb"

In [4]:
def check_if_files_present(test_list, slice_list):
    for test in test_list:
        for slice_i in slice_list:
            path = f"{dir}{test}/traj_1us_{slice_i}_PBC_fit.xtc"
            if not os.path.isfile(path):
                print(f"File {path} not found")
                return False
    return True

In [5]:
check_if_files_present(test_list, [1,2,3,4])

True

In [6]:
def get_frame_analysis(dir, test, slice, ile):
    tf=None
    #tf=1000
    molsys=DR.MolSys(pdb_file_path, f"{dir}{test}/traj_1us_{slice}_PBC_fit.xtc",tf=tf)
    select=DR.MolSelect(molsys)

    frames=list()
    frames.append({'Type':'methylCC','Nuc':'ivl', 'resids':ile,'sigma':0})
    frames.append({'Type':'side_chain_chi','Nuc':'ivl', 'resids':ile,'n_bonds':1,'sigma':0})
    frames.append({'Type':'side_chain_chi','Nuc':'ivl', 'resids':ile,'n_bonds':2,'sigma':0})

    "Frames with post-process smoothing"
    frames.append({'Type':'hops_3site','Nuc':'ivl', 'resids':ile,'sigma':5})
    frames.append({'Type':'methylCC','Nuc':'ivl', 'resids':ile,'sigma':5})
    frames.append({'Type':'chi_hop','Nuc':'ivl', 'resids':ile,'n_bonds':1,'sigma':50})
    frames.append({'Type':'side_chain_chi','Nuc':'ivl', 'resids':ile,'n_bonds':1,'sigma':50})
    frames.append({'Type':'chi_hop','Nuc':'ivl', 'resids':ile,'n_bonds':2,'sigma':50})
    frames.append({'Type':'side_chain_chi','Nuc':'ivl', 'resids':ile,'n_bonds':2,'sigma':50})

    select.select_bond(Nuc='ivl', resids=ile)
    fr_obj=DR.Frames.FrameObj(select)
    fr_obj.tensor_frame(sel1=1,sel2=2)
    for f in frames:fr_obj.new_frame(**f)
    fr_obj.load_frames(n=-1)
    fr_obj.post_process()

    tf = molsys._traj.tf
    print(f"tf: {tf}")

    include=np.zeros([3,9],dtype=bool)
    include[0][:3]=True  
    include[1][[4,6,8]]=True
    include[2][3:]=True
    
    t=np.arange(tf>>1)*.005
    data=fr_obj.frames2data(mode='full')
    proj=Project(f"{dir}{test}/{slice}.nosync",create=True)
    for d in data:proj.append_data(d)
    proj[0].detect.r_no_opt(10)
    proj.fit(bounds=False)
    proj['no_opt'][0].detect.r_auto(5)
    proj['no_opt'].fit()
    #proj.save()
    return [fr_obj.frames2ct(include=x, mode='full') for x in include]
    

In [7]:
#uncomment for testing
#out = get_frame_analysis(dir, test_list[0], 1, [3])

In [8]:
print("starting")
print(f"number of tests: {len(test_list)}")
print(f"number of slices: {4}")
print(f"number of ile_resn: {len(ile_resn)}")
total_number_of_jobs = len(test_list) * 4 * len(ile_resn)
print(f"total number of jobs: {total_number_of_jobs}")
job_idx = 0
for test in test_list:
    for slice in range(1,5):
        for ile in ile_resn:
            job_idx += 1
            print(f"job % {job_idx/total_number_of_jobs*100:.2} \t {test} {slice} {ile}", flush=True)
            try:
                out_pickle_path = f"{dir}{test}/{slice}_{ile}.p"
                if os.path.exists(out_pickle_path):
                    #print(f"skipping {out_pickle_path}", flush=True)
                    continue
                else:
                    print(f"creating {out_pickle_path}", flush=True)
                    out = get_frame_analysis(dir, test, slice, [ile])
                    pickle.dump(out, open(out_pickle_path, "wb"))
            except Exception as e:
                print(e)
                pass

starting
number of tests: 8
number of slices: 4
number of ile_resn: 7
total number of jobs: 224
job % 0.45 	 test_001 1 3
job % 0.89 	 test_001 1 13
job % 1.3 	 test_001 1 23
job % 1.8 	 test_001 1 30
job % 2.2 	 test_001 1 36
job % 2.7 	 test_001 1 44
job % 3.1 	 test_001 1 61
job % 3.6 	 test_001 2 3
job % 4.0 	 test_001 2 13
job % 4.5 	 test_001 2 23
job % 4.9 	 test_001 2 30
job % 5.4 	 test_001 2 36
job % 5.8 	 test_001 2 44
job % 6.2 	 test_001 2 61
job % 6.7 	 test_001 3 3
job % 7.1 	 test_001 3 13
job % 7.6 	 test_001 3 23
job % 8.0 	 test_001 3 30
job % 8.5 	 test_001 3 36
job % 8.9 	 test_001 3 44
job % 9.4 	 test_001 3 61
job % 9.8 	 test_001 4 3
job % 1e+01 	 test_001 4 13
job % 1.1e+01 	 test_001 4 23
job % 1.1e+01 	 test_001 4 30
job % 1.2e+01 	 test_001 4 36
job % 1.2e+01 	 test_001 4 44
job % 1.2e+01 	 test_001 4 61
job % 1.3e+01 	 test_002 1 3
job % 1.3e+01 	 test_002 1 13
job % 1.4e+01 	 test_002 1 23
job % 1.4e+01 	 test_002 1 30
job % 1.5e+01 	 test_002 1 36
job % 1

In [9]:
"/home/mlehner/pyDR/1ubq_data/test_001/traj_1us_1_PBC_fit.xtc"

'/home/mlehner/pyDR/1ubq_data/test_001/traj_1us_1_PBC_fit.xtc'